This is a notes of Kitaev spin liquid under a local magnetic field. The code was written in real space with maximum $80 \times 80 \times 2$ sites. 

\section{Hamiltonian}

\begin{align}
    H &= H_K + H_{h} \\
    H_K &= - \sum_{<j,k>_\alpha} J_\alpha \sigma_j^\alpha \sigma_k^\alpha \\
    H_h &= - \sum_\alpha h_\alpha \sigma_0^\alpha
\end{align}

The ground state Hamiltonain of Kitaev model is :
\begin{equation}
    H_K^0 = \sum_{<j,k>_\alpha} J_\alpha \ i c_j c_k
\end{equation}

When applied the local magnetic field, the $<u_{<0j>_\alpha}$ is not conserved anymore while others are still conserved quanties. We can rewrite the Hamiltonian into this form:

\begin{align}
\begin{split}
    H &= H_K^0 + \sum_{<0j>_\alpha} J_\alpha \ (i b_0^\alpha b_j^\alpha - 1) \ i c_0 c_j
            - \sum_\alpha h_\alpha \ i c_0 b_0^\alpha \\
      & = H_K^0 + H_{int} + H_h\\
      & = H_K^0 + H_{imp}
\end{split}
\end{align}

where $H_{imp} = H_{int} + H_h$.


\section{MFT Hamiltonian}

Apply MFT to $H_{int}$, we have:
\begin{equation}
    H_{int}^{MF} = \sum_\alpha J_\alpha ( <i c_0 c_\alpha> i b_0^\alpha b_\alpha^\alpha  
                        + （ <i b_0^\alpha b_\alpha^\alpha> -1） i c_0 c_\alpha
                        - <i c_0 b_0^\alpha> i c_\alpha b_\alpha^\alpha
                        - <i c_\alpha b_\alpha^\alpha> i c_0 b_0^\alpha)
\end{equation}
The constant term is:
\begin{equation}
    H_{int}^{C} = \sum_\alpha J_\alpha( <i c_0 b_0^\alpha> <i c_\alpha b_\alpha^\alpha>
                                    - <i c_0 c_\alpha><i b_0^\alpha b_\alpha^\alpha>)
\end{equation}

where we use superscript $\alpha$ to label bonds direction and subscript $\alpha$ to label atoms link to the $0$-atom with bond $\alpha$. And we ignore the $(i c_0 b_\alpha^\alpha)(i b_0^\alpha c_\alpha)$ terms because they have been proved $0$ in previous studies.

There should be a local constrain on site-$0$: $i c_0 b_0^x b_0^y b_0^z =1$. It is applied as Lagrange multipliers:
\begin{equation}
    H_{lag} = \sum_\alpha \lambda_\alpha ( i c_0 b_0^\alpha 
                    + \epsilon_{\alpha \beta \gamma} i b_0^\beta b_0^\gamma)
\end{equation}

\section{Lattice}

We use a $N_1 \times N_2 \times 2$ rhomboid lattice. The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label each unit cell by its row and colume numbers $(r, c)$(start from $0$), the label of this unit cell given by $ n = r \times N_1 + c$. We can label A atom in the $n$-th unit cell as $2n$ and B atom as $2n + 1$.

For a given A atom $2n$, the B atom linked to it by $x$-bond is $2n + 1$, $y$-bond is $2n - 1$, $z$-bond is $2 (n - N_1) + 1$.

we use the basis $(c_{1A},\ c_{1B},\ ... \,\ c_{NA},\ c_{NB})^T$ when there is no magnetic field. When $H_h$ is added, the basis dimension becomes $N_1 \times N_2 \times 2 + 6$,  where the last six operators in the basis is $(b_0^x,\ b_0^y,\ b_0^z,\ b_x^x,\ b_y^y,\ b_z^z)$.

In [186]:
import numpy as np
from numpy.linalg import eigh
from scipy.sparse import csc_matrix

def BulkHam(JK, N1, N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: H matrix'''
    
    SiteNum = N1 * N2 * 2
    Hr = np.zeros([SiteNum, SiteNum],dtype=complex)
    for r in range(N2):
        for c in range(N1):
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2 * n + 1
            Hr[2*n, next] +=  1j * JK[0]  
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                Hr[2*n, next] += 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if flux == True and r == N2//2 and c >= N1//2:
                    Hr[2*n, next] +=  -1j * JK[2]
                else:
                    Hr[2*n, next] +=  1j * JK[2]
                    
    Hr = (Hr + np.conj(Hr).T)/2
    return Hr

def BulkHam_csc(JK, N1, N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: positions and values of non-zero elements of HK^0'''
    
    SiteNum = N1 * N2 * 2
    
    row = []
    col = []
    data = []
    for r in range(N2):
        
        for c in range(N1):
            temp = {} # A temporary dict
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2*n + 1
            if next in temp:
                temp[next] += 1j * JK[0]
            else: 
                temp[next] = 1j * JK[0]
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                if next in temp:
                    temp[next] += 1j * JK[1]
                else: 
                    temp[next] = 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if next in temp:
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] +=  -1j * JK[2]
                    else:
                        temp[next] +=  1j * JK[2]
                else: 
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] =  -1j * JK[2]
                    else:
                        temp[next] =  1j * JK[2]
            odTemp = sorted(temp)
            for e in odTemp:
                row.append(2*n)
                col.append(e)
                data.append(temp[e])
                    
    return row, col, data

def ImpHam_csc(JK, N1, N2, h, param):
    '''H_imp = H_int + H_h
    
       Inputs: h:1*3 matrix, hx=h[0], hy = h[1], hz=h[2]
               param[0:2]: i c0 b0^a , a = x,y,z
               param[3:5]: i ca ba^a , a = x,y,z
               param[6:8]: i c0 ca , a = x,y,z
               param[9:11]: i b0^a ba^a , a = x,y,z
               param[12:14]: Lagrange multipliers
               
       Return: positions and values of non-zero elements of H_imp'''
    # position of the impurity site
    imp = N1//2 * N1 + N2//2 
    
    # positions of atoms linked to the impurity
    next = np.array([2*imp +1, 2*imp -1, 2*(imp - N1) +1]) 
        
    SiteNum = N1 * N2 * 2
    row = []
    col = []
    data = []
    
    
    '''Effective magnetic field of i c0 b0^a'''
    heff = np.zeros(3, dtype = complex)
    for i in range(3):
        heff[i] = 1j * h[i] + 1j * param[12 + i] - 1j * JK[i]* param[3 + i]
        row.append( 2*imp )
        col.append( SiteNum + i )
        data.append(heff[i])

    '''H_int : JK[a] <i c0 b0^a> i ca ba^a'''
    for i in range(3):
        row.append( next[i] )
        col.append( SiteNum + 3 + i )
        data.append( - 1j * JK[i]* param[i]  )  
        
    '''H_int : JK[a] <i c0 ca> i b0^a ba^a'''
    for i in range(3):
        row.append( SiteNum + i )
        col.append( SiteNum + 3 + i )
        data.append( 1j * JK[i]* param[6 + i] )
        
    '''H_int : （ JK[a] <i b0^a ba^a>  -1） i c0 ca'''
    for i in range(3):
        row.append( 2*imp )
        col.append( next[i] )
        data.append( 1j * JK[i] * ( param[9 + i] - 1))
        
    ''' H_la : i b0^b ib^c'''
    for i in range(3):
        row.append( SiteNum + (i+1)%3 )
        col.append( SiteNum + (i+2)%3 )
        data.append( 1j * param[12 + i])

    return row, col, data

def makeH(row, col, data, dim):
    '''Make csc format Hamiltonian
       
       Inputs : row,col :position of the non-zero elementds of Hamiltonian matrix
                data: H(row,col) = data
                dim: Hamiltonian matrix dimension
       Return: Hcsc: csc formate Hamiltonian
       
       Notice: we only have to enter half of the elements and get the other half 
               by hermition conjugate.'''
    Hcsc = csc_matrix((data, (row, col)), shape=(dim, dim),dtype = complex)
    Hcsc = (Hcsc + Hcsc.getH()) /2 
    return Hcsc

def InitParameters(m0 = 0, ma = 0, cc = -0.52489267, ua = 1, la = 0):
    '''initiate parameters'''
    param = np.zeros(15)
    for i in range(3):
        param[i] += m0 
        param[3 + i] += ma 
        param[6 + i] += cc 
        param[9 + i] += ua 
        param[12 + i] += la
    return param


\section{Equal time Green's Functions of Majorana fermions}
\quad Once we have the Hamiltonian matrix, we proceed to calculate the order parameters. Suppose we already know the eigen values $\epsilon_j$ and corresponding eigen vectors $| \psi_j \rangle$ of a fermionic Hamiltonian $\hat H$. The Green's function is:
\begin{align}
\begin{split}
    \hat{G}(i \omega_n ) &=  \frac{1}{ i \omega_n - \hat{H}} \\
           &=  \frac{1}{ i \omega_n - \hat{H}} \sum_j | \psi_j \rangle \langle \psi_j |\\
           & =\sum_j \frac{| \psi_j \rangle \langle \psi_j |}{i \omega_n - \epsilon_j}
\end{split}
\end{align}

\begin{align}
\begin{split}
    \hat{G}(0^- ) &=  \frac{1}{ \beta } \sum_{i \omega_n} \hat{G}(i \omega_n ) 
                       e^{ i \omega_n 0^-}\\
           &=  - \int_C \frac{\rm{d} \it z}{2 \pi i } f(z) G(z) e^{z 0^-}\\
           & =\sum_j | \psi_j \rangle \langle \psi_j | f( \epsilon_j)
\end{split}
\end{align}

where $f(\epsilon)$ is the Fermi distribution. Then we have:

\begin{equation}
    [\hat{G}(0^-]_{mn} = \sum_j | \psi_j\rangle_m \langle \psi_j |_n f(\epsilon_l)
\end{equation}

\quad For Majorana fermions, we can transform it to complex fermions by a unitary transformation: 

\begin{equation}
    c = \sqrt{2} U \psi, c^t = \sqrt{2} \psi^\dagger U^\dagger
\end{equation}

There thus be a overall factor $2$ in the Hamitonian density, i.e. $ h_{Maj} = 2 h_{Fermi}$. And another factor $2$ in the correlations, i.e. $<i c^t c> = 2 i <f^\dagger f>$. In the following code, we can change $ H \rightarrow 2H$ and $f( \epsilon_j) \rightarrow 2 f( \epsilon_j)$ to get the correct correlations.

In [215]:
'''PLEASE RUN THE 1st CODE SECTOR BEFORE THIS ONE.'''
'''Use scipy.optimize.root() to solve order parameters'''
from scipy.optimize import root

def Fermi(eng, T):
    '''Fermi distribution function
       Inputs: eng : Energy
               T : Temperature '''
    fermi = 1 / (1 + np.exp(eng / T) )
    return fermi

def Gfs(N1, N2, Harr):
    '''Calculate the equal time Green's functions of order parameters.
       
       Inputs: N1, N2 : number of unit cells in two directions
              Harr: Hamiltonian array
                      
       Return: Nparam: new parameters'''
    
    Nparam = np.zeros(15, dtype = complex) 
    
    # eigen values and vectors of H
    Harr *= 2
    e, v = eigh(Harr, 'L')
    mask = e < 0
    u = v[:,mask].T
    ak = np.conj(u).T @ u
    
    # position of the impurity site
    imp = N1//2 * N1 + N2//2   
    # positions of atoms linked to the impurity
    next = np.array([2*imp +1, 2*imp -1, 2*(imp - N1) +1]) 
    SiteNum = N1 * N2 * 2 
    
    for i in range(3):
        '''i c0 b0^a'''
        Nparam[i] += ak[2*imp,SiteNum +i] * 1j * 2
            
        '''i ca ba^a'''
        Nparam[3 + i] += ak[next[i],SiteNum +3 +i] * 1j * 2
            
        '''i c0 ca'''
        Nparam[6 + i] += ak[2*imp,next[i]] * 1j * 2
            
        '''i b0^a ba^a'''
        Nparam[9 + i] += ak[SiteNum +i,SiteNum +3 +i] * 1j * 2

        '''lagrange multipliers'''
        Nparam[12 + i] += Nparam[i]
        Nparam[12 + i] += ak[SiteNum+(i+1)%3,SiteNum+(i+2)%3] * 1j * 2
    if all(Nparam.imag < 1.e-10):
        return Nparam.real
    else:
        return Nparam

def Gfs_slow(N1, N2, Harr):
    '''Calculate the equal time Green's functions of order parameters.
       This code is slow however explicity write the calculation formulas
       
       Inputs: N1, N2 : number of unit cells in two directions
              Harr: Hamiltonian array
                      
       Return: Nparam: new parameters'''
    
    Nparam = np.zeros(15, dtype = complex) 
    
    # eigen values and vectors of H
    Harr *= 2
    e, v = eigh(Harr, 'L')
    cv = np.conj(v)
    
    #v = v.T
    #cv = cv.T
    
    # position of the impurity site
    imp = N1//2 * N1 + N2//2   
    # positions of atoms linked to the impurity
    next = np.array([2*imp +1, 2*imp -1, 2*(imp - N1) +1]) 
    SiteNum = N1 * N2 * 2 
    
    Num = len(e)
    for l in range(Num//2):
        fermi = 2* Fermi(e[l], T)
        for i in range(3):
            '''i c0 b0^a'''
            Nparam[i] += cv[2*imp,l] * v[SiteNum +i,l] * 1j * fermi
            
            '''i ca ba^a'''
            Nparam[3 + i] += cv[next[i],l] * v[SiteNum +3 +i,l] * 1j * fermi
            
            '''i c0 ca'''
            Nparam[6 + i] += cv[2*imp,l] * v[next[i],l] * 1j * fermi
            
            '''i b0^a ba^a'''
            Nparam[9 + i] += cv[SiteNum +i,l] * v[SiteNum +3 +i, l] * 1j * fermi

            '''lagrange multipliers'''
            Nparam[12 + i] += Nparam[i]
            Nparam[12 + i] += cv[SiteNum+(i+1)%3,l] * v[SiteNum+(i+2)%3,l] * 1j * fermi
    if all(Nparam.imag < 1.e-10):
        return Nparam.real
    else:
        return Nparam
    
def Iteration(error, dt):
    return 

In [217]:
'''test code'''
# System setups
JK = np.array([1,1,1])
T = 1.e-5

N1 = 17
N2 = 17
SiteNum = N1 * N2 * 2
dim = SiteNum + 6

h = np.array([0, 0, 0])
param = InitParameters()

# Generate bulk Hamiltonian
row = []
col = []
data = []
row, col, data = BulkHam_csc(JK, N1,N2,periodic = True)

# Generate impurity Hamiltonian
rowImp = []
colImp = []
dataImp = []
rowImp, colImp, dataImp = ImpHam_csc(JK, N1,N2, h, param)

Hcsc = makeH(row, col, data, dim) + makeH(rowImp, colImp, dataImp, dim)
Harr = Hcsc.toarray()

Nparam = Gfs(N1,N2, Harr)
print(Nparam)


[ 0.          0.          0.          0.          0.          0.
 -0.52489267 -0.52489267 -0.52489267  1.          1.          1.
  0.          0.          0.        ]
